# Семинар 1

## Введение в TF-IDF

TF-IDF (частота термина — обратная частота документа) — это статистическая мера, используемая для оценки важности слова в контексте документа, который является частью коллекции или корпуса. Эта величина увеличивается пропорционально количеству раз, когда слово появляется в документе, но компенсируется частотой слова в корпусе, что помогает контролировать факт, что некоторые слова встречаются более обще.

## Ключевые концепции

- **Частота термина (TF)**: Количество раз, когда определенное слово появляется в документе.
- **Обратная частота документа (IDF)**: Мера того, насколько уникально или редко слово встречается во всем корпусе.
- **Зачем использовать TF-IDF**: Этот метод помогает выявлять наиболее важные слова в тексте, что особенно полезно для задач обработки естественного языка, таких как поиск информации, классификация текстов и других.

В этом разделе мы подробно рассмотрим, как рассчитывается TF-IDF, и покажем, как его можно применять на практике для анализа текстовых данных.

$$\text{tf-idf}(t, d, D) = \text{tf}(t, d) \times \text{idf}(t, d, D)$$

Здесь  
t - обозначает конкретные слова;  
d - обозначает каждый документ;  
D - обозначает коллекцию документов

Первая часть формулы $tf(t, d)$ отвечает за частоту слов, которая определяется как количество раз, которое втстречается конкретное слово $t$ в рассматриваемом документе $d$.

Чтобы понять вторую часть формулы $\text{idf}(t, d, D)$, обратной частоты документа, давайте сначала запишем полную математическую формулу для IDF.

$$ idf(t, d, D) = log \frac{ \mid \text{ } D \text{ } \mid }{ 1 + \mid \{ d : t \in d \} \mid } $$

Числитель: D относится к нашему пространству документов. Его также можно представить как D = ${ d_{1}, d_{2}, \dots, d_{n} }$, где n - количество документов в вашей коллекции.

Знаменатель: $\mid { d: t \in d } \mid$ - это частота документов. Или по другому, это количество документов $d$, содержащих слово $t$. Обратите внимание, что это подразумевает, что не имеет значения, появился ли термин 1 раз или 100 раз в документе, он все равно будет учитываться как 1, поскольку слово действительно появилось в документе.  
Также будем считать, что если слово встретилось во всех документах, то его $$ idf(t, d, D) = 0 $$

Константа 1 добавляется к знаменателю, чтобы избежать ошибки деления на ноль, если термин не содержится ни в одном документе тестового набора данных.

### Задание 1

Посчитаем на игрушечном примере

In [1]:
import numpy as np
from collections import Counter
from math import log

Напишем класс для подсчета TF-IDF заданного корпуса документов (предложений). Вам нужно реализовать два метода, вычисляющие TF и IDF. Подсказки по реализации можно найти в описании методов

In [2]:
class ToyTfIdf:
    def __init__(self, sentences):
        self.sentences = [self.preprocess(sentence) for sentence in sentences]
        self.tf_scores = []
        self.idf_scores = {}
        self.tf_idf_scores = []

    @staticmethod
    def preprocess(sentence: str) -> str:
        return sentence.lower().replace('.', '')

    def compute_tf(self, sentence: str) -> dict[str, float]:
        """
        Computes the term frequency for each word in a given sentence.
        Recall that Term Frequency (TF) for a word in a document is
        calculated by dividing the number of times the word appears in the document by the total number of words in the document.
        """
        # YOUR CODE HERE
        pass

    def compute_idf(self):
        """
        Calculates the inverse document frequency for each unique word across all sentences.
        Inverse Document Frequency (IDF) measures how important a word is across multiple documents.
        It's calculated as the logarithm of the ratio of the total number of documents to the number of documents containing the word.
        """
        # YOUR CODE HERE
        pass

    def compute_tf_idf(self):
        """
        Calculates the TF-IDF score for each word in each sentence based on the previously computed TF and IDF scores.
        """
        # YOUR CODE HERE
        pass

    def calculate_scores(self) -> list[dict[str, float]]:
        """
        Orchestrates the computation of TF, IDF, and TF-IDF scores for the given sentences.
        """
        # Calculate TF scores
        self.tf_scores = [self.compute_tf(sentence) for sentence in self.sentences]
        self.compute_idf()
        self.compute_tf_idf()

        return self.tf_idf_scores

# Example usage
toy_data = [
    "Кот спит на подушке.",
    "Собака лежит на ковре в гостиной.",
    "Кот и собака играют вместе на улице."
]

toy_tf_idf = ToyTfIdf(toy_data)
tf_idf_results = toy_tf_idf.calculate_scores()
tf_idf_results

[]

In [3]:
# solved

class ToyTfIdf:
    def __init__(self, sentences):
        self.sentences = [self.preprocess(sentence) for sentence in sentences]
        self.tf_scores = []
        self.idf_scores = {}
        self.tf_idf_scores = []

    @staticmethod
    def preprocess(sentence: str) -> str:
        # Lowercase and remove punctuation
        return sentence.lower().replace('.', '')

    def compute_tf(self, sentence: str) -> dict[str, float]:
        tf_dict = {}
        words = sentence.split()
        word_count = len(words)
        word_counts = Counter(words)
        for word, count in word_counts.items():
            tf_dict[word] = count / float(word_count)
        return tf_dict

    def compute_idf(self):
        N = len(self.sentences)
        all_words = set(word for sentence in self.sentences for word in sentence.split())
        for word in all_words:
            count = sum(word in sentence.split() for sentence in self.sentences)
            if count == len(self.sentences):
                self.idf_scores[word] = 0
            else:
                self.idf_scores[word] = log(N / (1 + float(count)))

    def compute_tf_idf(self):
        for tf in self.tf_scores:
            tf_idf_dict = {}
            for word, tf_score in tf.items():
                tf_idf_dict[word] = tf_score * self.idf_scores[word]
            self.tf_idf_scores.append(tf_idf_dict)

    def calculate_scores(self) -> list[dict[str, float]]:
        # Calculate TF scores
        self.tf_scores = [self.compute_tf(sentence) for sentence in self.sentences]

        # Calculate IDF scores
        self.compute_idf()

        # Calculate TF-IDF scores
        self.compute_tf_idf()

        return self.tf_idf_scores


sentences = [
    "Кот спит на подушке",
    "Собака лежит на ковре в гостиной",
    "Кот и собака играют вместе на улице"
]

toy_tf_idf = ToyTfIdf(sentences)
tf_idf_results = toy_tf_idf.calculate_scores()
tf_idf_results

[{'кот': 0.0,
  'спит': 0.1013662770270411,
  'на': 0.0,
  'подушке': 0.1013662770270411},
 {'собака': 0.0,
  'лежит': 0.06757751801802739,
  'на': 0.0,
  'ковре': 0.06757751801802739,
  'в': 0.06757751801802739,
  'гостиной': 0.06757751801802739},
 {'кот': 0.0,
  'и': 0.05792358687259491,
  'собака': 0.0,
  'играют': 0.05792358687259491,
  'вместе': 0.05792358687259491,
  'на': 0.0,
  'улице': 0.05792358687259491}]

In [5]:
toy_tf_idf = ToyTfIdf(["Кот спит на подушке"])
computed_tf = toy_tf_idf.compute_tf(toy_tf_idf.sentences[0])
expected_tf_for_word_кот = 0.25  # The word 'Кот' appears once in a sentence of 4 words.
assert computed_tf.get('кот', 0) == expected_tf_for_word_кот, "TF calculation is incorrect for word 'кот'"

toy_tf_idf = ToyTfIdf(["Кот спит", "Кот ест", "Кот играет"])
toy_tf_idf.compute_idf()
expected_idf_for_common_word = 0  # The word 'Кот' appears in all 3 documents.
assert toy_tf_idf.idf_scores.get('кот', -1) == expected_idf_for_common_word, "IDF calculation is incorrect for a word appearing in all documents"

toy_tf_idf = ToyTfIdf(["Кот спит на подушке", "Собака лежит", "Кот и собака играют"])
toy_tf_idf.tf_scores = [toy_tf_idf.compute_tf(sentence) for sentence in toy_tf_idf.sentences]
toy_tf_idf.compute_idf()
toy_tf_idf.compute_tf_idf()
expected_tf_idf_for_word_спит = toy_tf_idf.tf_scores[0]['спит'] * toy_tf_idf.idf_scores['спит']
assert toy_tf_idf.tf_idf_scores[0].get('спит', 0) == expected_tf_idf_for_word_спит, "TF-IDF calculation is incorrect for word 'спит'"

toy_tf_idf = ToyTfIdf(["Кот спит", "Собака лежит", "Кот играет"])
result = toy_tf_idf.calculate_scores()
assert len(result) == 3, "The number of calculated TF-IDF dictionaries should match the number of sentences"
assert all(isinstance(d, dict) for d in result), "Each item in the result should be a dictionary"

print("All checks passed!")


All checks passed!


### Продолжаем

Отлично, теперь давайте посмотрим на пример реального использования TF-IDF из библиотеки sklearn для задачи кластеризации текстов

In [6]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Загрузим пример с данными из новостного источника Lenta.ru

In [7]:
!unzip ./lenta-ru-news_shrinked.zip

Archive:  ./lenta-ru-news_shrinked.zip
  inflating: lenta-ru-news_shrinked.csv  


In [8]:
data_path = './lenta-ru-news_shrinked.csv'

df = pd.read_csv(data_path)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92555 entries, 0 to 92554
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     92555 non-null  object
 1   title   92555 non-null  object
 2   text    92555 non-null  object
 3   topic   92555 non-null  object
 4   tags    85998 non-null  object
 5   date    92555 non-null  object
dtypes: object(6)
memory usage: 4.2+ MB


In [10]:
df['topic'].value_counts()

Спорт          64413
Дом            21734
Путешествия     6408
Name: topic, dtype: int64

In [11]:
df.loc[0, 'text']

'Семикратная победительница Уимблдона может на этой неделе отправиться в Антарктиду с тем, чтобы оттуда дать несколько прямых телеуроков игры в теннис. Соорудить подобие корта берется аргентинская антарктическая станция "Вице-коммондор Марамбио". Однако осуществить пожелание Штеффи Граф не так просто:станция находится в ведении аргентинского военного ведомства идоступ туда возможен только с разрешения правительства. Крометого, для полета на станцию необходим военно-транспортныйсамолет, способный совершить посадку на короткой снежнойполосе.'

In [12]:
wordpunct_tokenize(df.loc[0, 'text'])

['Семикратная',
 'победительница',
 'Уимблдона',
 'может',
 'на',
 'этой',
 'неделе',
 'отправиться',
 'в',
 'Антарктиду',
 'с',
 'тем',
 ',',
 'чтобы',
 'оттуда',
 'дать',
 'несколько',
 'прямых',
 'телеуроков',
 'игры',
 'в',
 'теннис',
 '.',
 'Соорудить',
 'подобие',
 'корта',
 'берется',
 'аргентинская',
 'антарктическая',
 'станция',
 '"',
 'Вице',
 '-',
 'коммондор',
 'Марамбио',
 '".',
 'Однако',
 'осуществить',
 'пожелание',
 'Штеффи',
 'Граф',
 'не',
 'так',
 'просто',
 ':',
 'станция',
 'находится',
 'в',
 'ведении',
 'аргентинского',
 'военного',
 'ведомства',
 'идоступ',
 'туда',
 'возможен',
 'только',
 'с',
 'разрешения',
 'правительства',
 '.',
 'Крометого',
 ',',
 'для',
 'полета',
 'на',
 'станцию',
 'необходим',
 'военно',
 '-',
 'транспортныйсамолет',
 ',',
 'способный',
 'совершить',
 'посадку',
 'на',
 'короткой',
 'снежнойполосе',
 '.']

In [13]:
tokenized_sentences = df['text'].apply(wordpunct_tokenize)
len(tokenized_sentences)

92555

### Давайте построим TF-IDF векторизатор для текста статей

В реальности вы не будете с нуля писать свой TF-IDF, поэтому давайте попробуем использовать готовую реализацию из библиотеки sklearn

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
import  nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['это', 'нею'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Initailize TfidfVectorizer

Инициализируйте TfidfVectorizer с ограничением размера словаря любым значением до 30000, игнорированием русских стоп-слов и использованием заданного токенизатора.

In [15]:
vectorizer = TfidfVectorizer(
    # YOUR CODE HERE
)

In [16]:
# solved

vectorizer = TfidfVectorizer(stop_words=russian_stopwords,
                             ngram_range=(1,3),
                             max_features=5024,
                             tokenizer=wordpunct_tokenize)

In [17]:
assert vectorizer.get_params()['stop_words'] == russian_stopwords, "You forgot to add russian stopwords for calculation"

assert vectorizer.get_params()['tokenizer'] == wordpunct_tokenize, "Tokenizer that you use is incorrect, it should be wordpunct_tokenize"

assert vectorizer.get_params()['max_features'] < 30000, "Size of your final vocabulary is inapropiate"

print("All checks passed!")

All checks passed!


In [18]:
matrix_tfidf = vectorizer.fit_transform(df['title'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [19]:
print(matrix_tfidf.shape)
print(type(matrix_tfidf))

(92555, 5024)
<class 'scipy.sparse._csr.csr_matrix'>


Матрица всех векторов для нашего корпуса документов имеет формат разрешенной матрицы для удобства хранения и скорости работы с данными, в случае необходимости вы можете преобразовать значения к привычным спискам или numpy array

In [20]:
matrix_tfidf[0].todense().shape

(1, 5024)

In [ ]:
dir(matrix_tfidf[0])

### Задание 2

Теперь, мы можем на основе имеющихся знаний построить простейшую систему поиска релевантных документов. В качестве меры близости будем использовать меру косинусной близости.

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

Построим простую систему поиска по статьям на основе нашего векторизатора, который мы создали ранее, только теперь организуем код в более структурном виде

На основе косинусной близости и векторизатора необходимо искать top_k наиболее похожих документа

In [23]:
class SimpleSearchEngine:
    def __init__(self, text_database: list[str]):
        self.raw_procesed_data = [self.preprocess(sample) for sample in text_database]
        self.base = []
        self.retriever = None
        self.inverted_index = {}
        self._init_retriever(text_database)
        self._init_inverted_index(text_database)

    @staticmethod
    def preprocess(sentence: str) -> str:
        return sentence

    def _init_retriever(self, text_database: list[str]):
        """
        TfidfVectorizer is used to convert a collection of raw documents into a matrix of TF-IDF features.
        Use fit_transform method of TfidfVectorizer to learn the vocabulary and idf from the training set and the transformed matrix.
        """
        # YOUR CODE HERE
        pass

    def retrieve(self, query):
        return self.retriever.transform([query])

    def retrieve_documents(self, query: str, top_k=3) -> np.array:
        """
        The query needs to be transformed into the same vector space as your document base.
        Utilize cosine_similarity to compute the similarity between the query vector and all document vectors in the base.
        Remember that cosine_similarity returns a matrix; you might need to flatten it to get a 1D array of similarity scores.
        Sort the documents based on their cosine similarity scores to find k most relevant ones to the query and return them as answer.
        """
        # YOUR CODE HERE
        pass

    def _init_inverted_index(self, text_database: list[str]):
        self.inverted_index = dict(enumerate(text_database))

    def display_relevant_docs(self, query: str, full_database, top_k=3) -> list[str]:
        docs_indexes = self.retrieve_documents(query, top_k=top_k)
        return [self.inverted_index[ind] for ind in docs_indexes]


simple_search_engine = SimpleSearchEngine(df['title'])
query = 'Какой отдых самый лучший?'

# simple_search_engine_results = simple_search_engine.display_relevant_docs(query, df['title'])
# simple_search_engine_results

In [24]:
# solved

class SimpleSearchEngine:
    def __init__(self, text_database: list[str]):
        self.raw_procesed_data = [self.preprocess(sample) for sample in text_database]
        self.base = []
        self.retriever = None
        self.inverted_index = {}
        self._init_retriever(text_database)
        self._init_inverted_index(text_database)

    @staticmethod
    def preprocess(sentence: str) -> str:
        return sentence

    def _init_retriever(self, text_database: list[str]):
        self.retriever = TfidfVectorizer(stop_words=russian_stopwords,
                             ngram_range=(1,3),
                             max_features=5024,
                             tokenizer=wordpunct_tokenize)
        self.base = self.retriever.fit_transform(text_database)

    def retrieve(self, query: str) -> np.array:
        return self.retriever.transform([query])

    def retrieve_documents(self, query: str, top_k=3) -> np.array:
        query_vector = self.retrieve(query)
        cosine_similarities = cosine_similarity(query_vector, self.base).flatten()
        relevant_indices = np.argsort(cosine_similarities, axis=0)[::-1][:top_k]
        return relevant_indices

    def _init_inverted_index(self, text_database: list[str]):
        self.inverted_index = dict(enumerate(text_database))

    def display_relevant_docs(self, query: str, full_database, top_k=3) -> list[str]:
        docs_indexes = self.retrieve_documents(query, top_k=top_k)
        return [self.inverted_index[ind] for ind in docs_indexes]


simple_search_engine = SimpleSearchEngine(df['title'])
query = 'Какой отдых самый лучший?'

simple_search_engine_results = simple_search_engine.display_relevant_docs(query, df['title'])
simple_search_engine_results

['Слепцовой предоставили отдых',
 'Любители Apple обошли пользователей Android по\xa0тратам на\xa0отдых',
 'Испанцы превратили отдых пенсионерки в\xa0катастрофу']

In [25]:
assert simple_search_engine.base.shape[0] == len(df['title']), "The number of rows in TF-IDF matrix should match the number of documents in the corpus"
assert simple_search_engine.base.shape[1] > 0, "The number of features in TF-IDF matrix should be greater than 0"

query_vector = simple_search_engine.retriever.transform([query])
assert query_vector.shape[1] == simple_search_engine.base.shape[1], "Query vector and document TF-IDF matrix should have the same number of features"

top_documents = simple_search_engine.retrieve_documents(query, top_k=3)
assert len(top_documents) <= 3, "The number of retrieved documents should not exceed the specified top_n"
assert all(0 <= doc_index < len(df['title']) for doc_index in top_documents), "Retrieved document indices must be valid within the corpus range"

cosine_similarities = cosine_similarity(query_vector, simple_search_engine.base).flatten()
assert all(sim >= 0 for sim in cosine_similarities), "Cosine similarities should be non-negative"

print("All checks passed!")

All checks passed!


Пример, как мы можем искать похожие документы

In [28]:
test_text = 'Какой отдых самый лучший?'

docs = simple_search_engine.retrieve_documents(test_text, top_k=5)
docs

array([43883, 62932, 89949, 72968, 88256])

In [29]:
df.loc[docs, 'title'].to_list()

['Слепцовой предоставили отдых',
 'Любители Apple обошли пользователей Android по\xa0тратам на\xa0отдых',
 'Испанцы превратили отдых пенсионерки в\xa0катастрофу',
 'Люди предпочли отдых в\xa0одиночестве общению с\xa0семьей и\xa0друзьями',
 'Сингапурцы запустят самый длинный авиарейс']

Как видите, система работает далеко не идеально, поэтому для более тонкой настройки необходимо придумывать метрики для оценки качества системы и итеративно улучшать данную реализацию

## Векторизация слов

### Продолжаем

Теперь, давайте перейдем к методам векторизации конкретных слов. Это может быть поллезно, если система которую вы строите имеет ограничения по производительности, а ваш домен достаточно специализирован.

Также этот метод крайне полезен при анализе ваших данных на этапе EDA

In [31]:
!pip install fasttext -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [32]:
from string import punctuation
from tqdm.auto import tqdm, trange
import fasttext

from multiprocessing import Pool

punkt = [p for p in punctuation] + ["`", "``" ,"''", "'"]

Давайте определим новую функцию для токенизации, которая будет исключать все слова, которые входят в определенный ранее список стоп-слов, и также будем игнорировать все слова, которые будут принадлежать списку для пунктуации.

Этап процессинга данных часто может занимать достаточно много времени, поэтому лучше выполнять его в параллельном режиме. Здесь можно использовать самый удобный способ распараллеливания - использовать Pool

In [33]:
def tokenize(sent: str) -> str:
    # YOUR CODE HERE
    pass

with Pool(8) as p:
    titles_preprocessed = list(
        # YOUR CODE HERE
                             )

In [34]:
# solved

def tokenize(sent: str) -> str:
        sent = word_tokenize(sent)
        return ' '.join([word for word in sent if word not in russian_stopwords and word not in punkt])

with Pool(8) as p:
    titles_preprocessed = list(tqdm(
                                    p.imap(tokenize, df['title'].tolist()),
                                    total=df['title'].size
                                    )
                             )

  0%|          | 0/92555 [00:00<?, ?it/s]

После того, как мы подготовили данные, можно приступать к обучению своего собственного векторизатора. Для начала мы обучим FastText - самый продвинутый из методов векторизации слов на основе семантической близости. Он позволяет решать проблему отсутствия некоторых слов в словаре за счет добавления буквенных n-gram при обучении представлений слов.

In [35]:
with open('titles_unsupervised.txt', 'w+', encoding='utf-8') as titles:
    for idx in range(len(titles_preprocessed)):
        titles.write(titles_preprocessed[idx]+'\n')

In [36]:
%%time
ft_vectors = fasttext.train_unsupervised('titles_unsupervised.txt', minn=3,maxn=5, dim=50)

CPU times: user 31.5 s, sys: 568 ms, total: 32 s
Wall time: 35.1 s


In [37]:
ft_vectors.get_nearest_neighbors('мяч')

[(0.9097076654434204, 'гол'),
 (0.8799793124198914, 'мяча'),
 (0.8685975670814514, 'головы'),
 (0.860861599445343, 'мячи'),
 (0.8484773635864258, 'забитый'),
 (0.8462731838226318, 'голос'),
 (0.8451036214828491, 'ворота'),
 (0.8438910841941833, 'голкипер'),
 (0.8364012837409973, 'ворот'),
 (0.8235505223274231, 'голове')]

In [38]:
ft_vectors.get_nearest_neighbors('мятч')

[(0.8021924495697021, 'мячи'),
 (0.7995634078979492, 'Полуголый'),
 (0.7913100719451904, 'гольфу'),
 (0.7897704839706421, 'голу'),
 (0.7894031405448914, '1:3'),
 (0.7858723402023315, 'Ницце'),
 (0.7855551838874817, '20-градусный'),
 (0.7849912047386169, 'забитый'),
 (0.7805249094963074, 'Шапекоэнсе'),
 (0.7796913981437683, '0:1')]

Как видите он может находить достаточно точные похожие слова, даже для опечаток

In [39]:
print(len(ft_vectors.words))
top3k = ft_vectors.words[:3000]
top3k[:10]

14436


['</s>', '«', '»', 'России', 'В', 'сборной', 'мира', 'Москве', 'матче', 'ЦСКА']

Обучим Word2vec на нашем корпусе текстов + добавим отдельный вектор для UNK, чтобы избежать вечной проблемы с поиском слов, которые отсутсвуют в тренировочной выборке.

Это может быть полезно, если вы используете данные вектора потом для обучения своей нейронной сети, а эти вектора будут инициализировать ваш слой Embedding

In [40]:
titles_for_w2v = [sent.split(" ") for sent in titles_preprocessed]

In [41]:
%%time
from gensim.models import Word2Vec
w2v = Word2Vec(sentences=titles_for_w2v, min_count=3, vector_size=50, window=6, seed=33, workers=4)

CPU times: user 6.28 s, sys: 73 ms, total: 6.35 s
Wall time: 4.13 s


In [42]:
w2v.save('lenta_titles_w2v_model.bin.gz')

In [43]:
w2v = Word2Vec.load('lenta_titles_w2v_model.bin.gz')
type(w2v)

gensim.models.word2vec.Word2Vec

In [ ]:
w2v.wv['мятч'].shape

In [45]:
w2v.wv.add_vector('UNK', np.random.uniform(low=-0.2, high=0.2, size=(50,)))

/usr/local/lib/python3.10/dist-packages/gensim/models/keyedvectors.py:551: UserWarning: Adding single vectors to a KeyedVectors which grows by one each time can be costly. Consider adding in batches or preallocating to the required size.
  warnings.warn(


21859

### Задание 3

Напишите класс для обучения своего Word2Vec. Реализуйте метод get_vector, который будет выдавать нужный вам вектор, а если вектора нет в словаре, тогда будет возвращать специальный вектор UNK

In [46]:
class CustomWord2Vec(Word2Vec):
    def __init__(self, *args, **kwargs):
        super(CustomWord2Vec, self).__init__(*args, **kwargs)
        self.unknown_word_vector = None # YOUR CODE HERE

    @staticmethod
    def text_preprocess(texts: list, tokenize_func) -> list[int]:
        with Pool(8) as p:
            texts_preprocessed = list()
        return texts_preprocessed

    def get_vector(self, word: str) -> np.array:
        # YOUR CODE HERE
        pass


In [47]:
# solved

class CustomWord2Vec(Word2Vec):
    def __init__(self, *args, **kwargs):
        super(CustomWord2Vec, self).__init__(*args, **kwargs)
        self.unknown_word_vector = np.random.uniform(low=-0.2, high=0.2, size=(50,))
        w2v.wv.add_vector('UNK', self.unknown_word_vector)

    @staticmethod
    def text_preprocess(texts: list[str], tokenize_func) -> list[int]:
        with Pool(8) as p:
            texts_preprocessed = list(tqdm(
                                            p.imap(tokenize_func, texts),
                                            total=df['title'].size
                                        )
                                     )
        return texts_preprocessed

    def get_vector(self, word: str) -> np.array:
        if word in self.wv.key_to_index:
            return self.wv[word]
        else:
            # Return the predefined vector for unknown words
            return self.unknown_word_vector


In [48]:
model = CustomWord2Vec(sentences=titles_for_w2v, min_count=3, vector_size=50, window=6, seed=33, workers=4)
model.get_vector("мятч")

array([-1.96007113e-01,  1.49344134e-01, -1.94199086e-01,  6.33644239e-02,
       -3.97968814e-02, -6.36083128e-02,  1.67107743e-01,  1.82702377e-01,
        1.58911811e-02, -3.82773050e-02, -9.26925351e-02, -6.33713117e-02,
       -1.63505821e-01, -5.64325614e-02,  1.24313401e-01, -1.39070786e-01,
       -1.55001213e-02, -1.39545678e-01,  2.15055431e-02,  6.55410365e-02,
        1.20170326e-01,  1.50698725e-04,  1.90471611e-01, -9.93446092e-02,
       -1.38637915e-01, -1.48869174e-01, -4.21020295e-02, -1.02278713e-01,
        1.08725416e-01, -1.98296565e-01,  9.40099524e-02,  1.78816081e-01,
        3.07701619e-02, -1.48359929e-01,  4.30810628e-02, -6.71934822e-03,
       -1.62762638e-01,  1.48280462e-01, -1.22372954e-01,  7.13991814e-02,
        1.02699252e-01,  3.52428301e-02, -1.33204720e-01,  1.90108664e-01,
       -5.59859387e-02, -6.26054870e-02, -9.44024341e-02,  1.71464825e-01,
        2.35067121e-04, -6.13968509e-02])

In [49]:
model.get_vector('UNK')

array([-1.96007113e-01,  1.49344134e-01, -1.94199086e-01,  6.33644239e-02,
       -3.97968814e-02, -6.36083128e-02,  1.67107743e-01,  1.82702377e-01,
        1.58911811e-02, -3.82773050e-02, -9.26925351e-02, -6.33713117e-02,
       -1.63505821e-01, -5.64325614e-02,  1.24313401e-01, -1.39070786e-01,
       -1.55001213e-02, -1.39545678e-01,  2.15055431e-02,  6.55410365e-02,
        1.20170326e-01,  1.50698725e-04,  1.90471611e-01, -9.93446092e-02,
       -1.38637915e-01, -1.48869174e-01, -4.21020295e-02, -1.02278713e-01,
        1.08725416e-01, -1.98296565e-01,  9.40099524e-02,  1.78816081e-01,
        3.07701619e-02, -1.48359929e-01,  4.30810628e-02, -6.71934822e-03,
       -1.62762638e-01,  1.48280462e-01, -1.22372954e-01,  7.13991814e-02,
        1.02699252e-01,  3.52428301e-02, -1.33204720e-01,  1.90108664e-01,
       -5.59859387e-02, -6.26054870e-02, -9.44024341e-02,  1.71464825e-01,
        2.35067121e-04, -6.13968509e-02])

В реальности вам все же лучше порой использовать предобученные вектора для слов, потому что чаще всего ваш корпус данных недостаточно репрезентативен, а задача только выиграет от интеграции общего доменного знания о представлении слов (дистрибутивная семантика)

Это можно сделать так

#### Осторожно, эти эмбединги достаточно большие и это может занять достаточно времени

In [ ]:
# https://fasttext.cc/docs/en/crawl-vectors.html

In [ ]:
# import fasttext.util
# fasttext.util.download_model('en', if_exists='ignore')  # fasttext.util.download_model('ru', if_exists='ignore') for russian
# ft = fasttext.load_model('cc.en.300.bin') # 'cc.ru.300.bin' for russian

In [ ]:
# open source analog for gensim wordvectors
# https://github.com/natasha/navec

In [ ]:
# import torch
# from slovnet.model.emb import NavecEmbedding

# emb = NavecEmbedding(navec)
# input = torch.tensor([1, 2, 0])
# output = emb(input)

# output.shape
# torch.Size([3, 300])

# output
# tensor([[ 4.2000e-01,  3.6666e-01,  1.7728e-01, -3.8719e-01, -1.0762e-01,
#           1.6954e-01, -4.6063e-01,  5.4519e-01, -2.1212e-01,  2.0965e-01,
#           1.9658e-01,  2.7807e-01, -2.3802e-01,  3.5155e-01,  1.4491e-02,
# 		  ...

## Контекстные вектора документов

### Продолжаем

Теперь перейдем к контекстуализированным эмбеддингам ваших документов/текстов с использованием предобученных Энкодерных текстовых моделей.
Для решения более 90% задач этот метод отлично подходит как одно из первых решений и будет если не первым, то уж вторым точно baseline` решением.

Напомним, что BERT-like модели хороши тем, что помимо того, что они возвращают контекстуализированный эмбеддинг для каждого токена, у них еще есть представление всего текста, которое можно доставать из CLS токена (почти всегда он представлен на первом месте)

Давайте попробуем векторизовать наши тексты с использованием легкой предобученной модели ruBERTtiny2. Она отлично подойдет для inferenca на CPU, потому что весит меньше 120 MB (для текстового энкодера это очень-очень мало).  
Также есть модель ruBERTtiny, которая еще меньше и весит всего 40 MB, но она чуть хуже на downstream-задачах

In [50]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
# model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text: str, model, tokenizer) -> np.array:
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('привет мир', model, tokenizer).shape)
# (312,)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

(312,)


### Задание 4

Векторизуйте все наши заголовки новостей и добавьте этот метод в написанный ранее класс SimpleSearchEngine

In [51]:
# solved
class BERTSearchEngine:
    def __init__(self, model, tokenizer, text_database):
        self.raw_procesed_data = [self.preprocess(sample, tokenizer) for sample in text_database]
        self.base = []
        self.retriever = None
        self.inverted_index = {}
        self._init_retriever(model, tokenizer, text_database)
        self._init_inverted_index(text_database)

    @staticmethod
    def preprocess(sentence: str, tokenizer):
        return tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')

    def _embed_bert_cls(self, tokenized_text: dict[torch.Tensor]) -> np.array:
        with torch.no_grad():
            model_output = self.retriever(**{k: v.to(self.retriever.device) for k, v in tokenized_text.items()})
        embeddings = model_output.last_hidden_state[:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings[0].cpu().numpy()

    def _init_retriever(self, model, tokenizer, text_database):
        self.retriever = model
        self.tokenizer = tokenizer
        self.base = np.array([self._embed_bert_cls(self.preprocess(text, tokenizer)) for text in tqdm(text_database)])

    def retrieve(self, query: str) -> np.array:
        return self._embed_bert_cls(self.preprocess(query, self.tokenizer))

    def retrieve_documents(self, query: str, top_k=3) -> list[int]:
        query_vector = self.retrieve(query)
        cosine_similarities = cosine_similarity([query_vector], self.base).flatten()
        relevant_indices = np.argsort(cosine_similarities, axis=0)[::-1][:top_k]
        return relevant_indices.tolist()

    def _init_inverted_index(self, text_database: list[str]):
        self.inverted_index = dict(enumerate(text_database))

    def display_relevant_docs(self, query, full_database, top_k=3) -> list[int]:
        docs_indexes = self.retrieve_documents(query, top_k=top_k)
        return [int(self.inverted_index[ind]) for ind in docs_indexes]


simple_search_engine = BERTSearchEngine(model, tokenizer, df['title'])
query = 'Какой отдых самый лучший?'

simple_search_engine_results = simple_search_engine.display_relevant_docs(query, df['title'])
simple_search_engine_results

  0%|          | 0/92555 [00:00<?, ?it/s]

['Названы лучшие для отдыха маяки',
 'Клубу "Что? Где? Когда?" пригрозили выселением из\xa0Нескучного сада',
 'Опубликовано лучшее туристическое видео года']

In [61]:
test_sentence = "Test sentence for preprocessing."
tokenized_data = BERTSearchEngine.preprocess(test_sentence, tokenizer)
test_embedding = simple_search_engine._embed_bert_cls(tokenized_data)

expected_embedding_shape = model.config.hidden_size
assert test_embedding.shape == (expected_embedding_shape,), f"The embedding should have shape ({expected_embedding_shape},)"

test_query = "Sample query for testing."
retrieved_docs_indices = simple_search_engine.retrieve_documents(test_query, top_k=5)

# Assert test for document retrieval
assert len(retrieved_docs_indices) <= 5, "retrieve_documents should return at most top_k documents"
assert all(isinstance(idx, int) for idx in retrieved_docs_indices), "All indices should be integers"

print("All checks passed!")

All checks passed!


У методов с использованием трансформерных моделей есть один существенный недостаток - у них есть ограничение на используемую длину контекста. Поэтому для обработки длинных текстов, у которых токенизированный контекст превышает заявленную максимальную длину приходится агрегировать кусочки контексты, разбитые на куски, которые помещаются в контекст.  
Стратегии агрегации и разбиения тут могут быть разные - с перекрытием, сплошное разбиение и любое другое, которое вы придумаете.

Давайте реализуем обработку длинного текста с разбитием на пересекающиеся участки. В качестве метода агрегации кусков контекста будет использовать усреднение векторов.

In [57]:
def split_into_chunks(data, chunk_size, overlap):
    # Calculate the number of chunks needed
    total_length = data['input_ids'].size(1)
    num_chunks = (total_length - overlap) // (chunk_size - overlap)
    if (total_length - overlap) % (chunk_size - overlap) != 0:
        num_chunks += 1

    # Create chunks with overlap for each tensor in the dictionary
    chunks = [
        # YOUR CODE HERE
    ]

    # Add an additional dimension to each tensor to maintain batch size of 1
    for chunk in chunks:
        for key in chunk:
            chunk[key] = chunk[key].unsqueeze(0)

    return chunks

In [58]:
# solved

def split_into_chunks(data, chunk_size, overlap):
    # Calculate the number of chunks needed
    total_length = data['input_ids'].size(1)
    num_chunks = (total_length - overlap) // (chunk_size - overlap)
    if (total_length - overlap) % (chunk_size - overlap) != 0:
        num_chunks += 1

    # Create chunks for each tensor in the dictionary
    chunks = [
        {
            key: data[key][0, i * (chunk_size - overlap):i * (chunk_size - overlap) + chunk_size]
            for key in data
        }
        for i in range(num_chunks)
    ]

    # Add an additional dimension to each tensor to maintain batch size of 1
    for chunk in chunks:
        for key in chunk:
            chunk[key] = chunk[key].unsqueeze(0)

    return chunks

In [60]:
tokenized_data = {'input_ids': torch.randn(1, 1000), 'token_type_ids': torch.randn(1, 1000), 'attention_mask': torch.randn(1, 1000)}
chunk_size = 512
overlap = 50

# Expected number of chunks
expected_num_chunks = (1000 - overlap) // (chunk_size - overlap)
if (1000 - overlap) % (chunk_size - overlap) != 0:
    expected_num_chunks += 1

chunks = split_into_chunks(tokenized_data, chunk_size, overlap)

assert len(chunks) == expected_num_chunks, "Number of chunks does not match the expected value"

for i, chunk in enumerate(chunks):
    if i < len(chunks) - 1:
        assert chunk['input_ids'].size(1) == chunk_size, f"Chunk {i} does not have the correct chunk size"
    else:
        # The last chunk can be smaller
        assert chunk['input_ids'].size(1) <= chunk_size, f"Last chunk {i} is larger than the specified chunk size"

if len(chunks) > 1:
    for i in range(len(chunks) - 1):
        end_of_first_chunk = chunks[i]['input_ids'][0, -overlap:]
        start_of_next_chunk = chunks[i+1]['input_ids'][0, :overlap]
        assert torch.equal(end_of_first_chunk, start_of_next_chunk), f"Chunks {i} and {i+1} do not overlap correctly"

print("All checks passed!")


All checks passed!


In [62]:
def bert_long_text_embedding(text, model, tokenizer, max_length=312, aggregation_mode='mean', chunk_size=256, overlap=64):
    # Split the text into chunks of max_length
    tokenized_text = tokenizer(text, return_tensors='pt')

    chunks = split_into_chunks(tokenized_text, chunk_size, overlap)

    # Process each chunk
    embeddings = []
    for chunk in chunks:
        with torch.no_grad():
            model_output = model(**{k: v.to(model.device) for k, v in chunk.items()})
        chunk_embedding = model_output.last_hidden_state[:, 0, :]
        chunk_embedding = torch.nn.functional.normalize(chunk_embedding)
        chunk_embedding = chunk_embedding[0].cpu()

        embeddings.append(chunk_embedding)

    # Aggregate embeddings
    if aggregation_mode == 'mean':
        return torch.mean(torch.stack(embeddings), dim=0)
    else:
        # Implement other aggregation strategies (e.g., max, concat) as needed
        raise NotImplementedError("Aggregation mode not implemented")

# Example usage
text = df['text'].sample().tolist()

embedding = bert_long_text_embedding(text, model, tokenizer)
embedding.shape

torch.Size([312])